# Ceneo Scraper

## Struktura opinii w serwisie ceneo.pl

|składowa|zmienna|selektor|
|--------|-------|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autora|author|.user-post__author-name|
|rekomendacje|recommandation|.user-post__author-recomendation > em|
|liczbę gwiazdek|rating|.user-post__score-count|
|treść opinii|content|.user-post__text|
|listę zalet|pros|review-feature__title--positives ~ review-feature__item|
|listę wad|cons|review-feature__title--negatives ~ review-feature__item|
|dawa wystawienia|opinion_date|.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|likes|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|dislikes|button.vote-no > span|

In [13]:
selectors = {
    'author': (".user-post__author-name",),
    'recommandation': (".user-post__author-recomendation > em",),
    'rating': (".user-post__score-count",),
    'content': (".user-post__text",),
    'pros': ("div.review-feature__title--positives ~ review-feature__item", None, True),
    'cons': ("div.review-feature__title--negatives ~ review-feature__item", None, True),
    'opinion_date': (".user-post__published > time:nth-child(1)", 'datetime'),
    'purchase_date': (".user-post__published > time:nth-child(2)", 'datetime'),
    'likes': ("button.vote-yes > span",),
    'dislikes': ("button.vote-no > span",),
    'opinion_id': (None, "data-entry-id"),
}

1. Import bibliotek

In [14]:
import os
import json
import requests
from bs4 import BeautifulSoup
from flask import Flask, render_template, request, redirect, url_for, session
import pandas as pd

2. Funkcja do ekstrakcji zawartości ze strony HTML

In [15]:
def extract(ancestor, selector, attribute = None, return_list = False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()

3. Adres pierwszej strony z opiniami o produkcie

In [16]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'loki'

@app.route('/')
@app.route('/home')
def home_page():
    return render_template('home.html')

# strona do ekstrakcji opinii
@app.route('/opinions', methods=['GET', 'POST'])
def opinions_page():
    data = []
    product_id = None
    error_message = None

    # pobranie ID z formularza
    if request.method == 'POST':
        product_id = request.form['product_id']
        if not product_id:
            error_message = 'Nie wpisano kodu produktu'
        else:
            url = f'https://www.ceneo.pl/{product_id}#tab=reviews'
            all_opinions = []

            response = requests.get(url)
            if response.status_code != 200:
                error_message = f'Nie można pobrać opinii o produkcie "{product_id}" lub produkt nie posiada żadnych opinii. Sprawdź, czy produkt o ID "{product_id}" istnieje.'

        # ekstrkacja wszystkich opinii
        while url:
            response = requests.get(url)
            page = BeautifulSoup(response.text, 'html.parser')
            opinions = page.select('div.js_product-review')
            for opinion in opinions:
                single_opinion = {
                    key: extract(opinion, *value)
                    for key, value in selectors.items()
                }
                all_opinions.append(single_opinion)
            try:
                url = 'https://www.ceneo.pl' + extract(page, 'a.pagination__next', 'href')
            except TypeError:
                url = None
        
        # tworzenie pliku JSON z opiniami o produkcie
        if not error_message:
            if not os.path.exists('opinions'):
                os.mkdir('opinions')
            with open(f'opinions/{product_id}.json', 'w', encoding='UTF-8') as jf:
                json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
        
        # wczytanie pliku JSON
        if not error_message:
            with open(f'opinions/{product_id}.json', 'r', encoding='UTF-8') as jf:
                data = json.load(jf)

        # zmienna product_id dla sesji
        session['product_id'] = product_id
    
    return render_template('opinions_extract.html', product_id=product_id, data=data, error_message=error_message)

@app.route('/products')
def products_page():
    return render_template('products_list.html')

@app.route('/author')
def author_page():
    return render_template('author.html')

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jun/2024 03:45:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2024 03:46:00] "GET /opinions HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2024 03:46:04] "POST /opinions HTTP/1.1" 200 -


In [ ]:
#product_id = '84514582'
# product_id = input('Podaj kod produktu z Ceneo.pl: ')
# url = f'https://www.ceneo.pl/{product_id}#tab=reviews'
# print(url)

4. Pobranie opinii o produkcie

In [ ]:
# all_opinions = []
# while(url): 
#     response = requests.get(url)
#     page = BeautifulSoup(response.text, 'html.parser')
#     opinions = page.select('div.js_product-review')
#     for opinion in opinions:
#         single_opinion = {
#             key: extract(opinion, *value)
#                 for key, value in selectors.items()
#         }
#         all_opinions.append(single_opinion)
#     try:
#         url = 'https://www.ceneo.pl' + extract(page, 'a.pagination__next', 'href')
#     except TypeError:
#         url = None

5. Stworzenie folderu 'opinions' oraz pliku JSON zawierającego opinie

In [ ]:
# if not os.path.exists('opinions'):
#     os.mkdir('opinions')
# with open(f'opinions/{product_id}.json', 'w', encoding='UTF-8') as jf:
#     json.dump(all_opinions, jf, indent=4, ensure_ascii=False)